In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name and compute calass for your CPU cluster 
cpu_cluster_name = "biaggi-cluster"
desired_vm_size = "Standard_D2_V2" 

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=desired_vm_size,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import MedianStoppingPolicy

import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.1, 1),
        "--max_iter": choice(25, 50, 75, 100)
    }
)


# Specify a Policy
policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = '.',
              entry_script='train.py',
              compute_target=cpu_cluster_name)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
primary_metric_name="accuracy",
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE
hyperdrive_config = HyperDriveConfig(   hyperparameter_sampling=ps,
                                        policy=policy,
                                        primary_metric_name=primary_metric_name,
                                        primary_metric_goal=primary_metric_goal,
                                        max_total_runs=100,
                                        estimator = est,
                                        max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.core.experiment import Experiment
experiment = Experiment(ws, ws.name)
hyperdrive_run = experiment.submit(hyperdrive_config)

In [ ]:

from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

from azureml.data.dataset_factory import TabularDatasetFactory
dataset = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv",                                                          validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True,                                                         partition_format=None, support_multi_line=False, empty_as_string=False, encoding='utf8')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(   experiment_timeout_minutes=30,
                                task='classification',
                                primary_metric= 'AUC_weighted',
                                experiment_exit_score = 0.9984,
                                blocked_models = ['KNN','LinearSVM'],
                                enable_onnx_compatible_models=True,
                                training_data= dataset,
                                label_column_name='y',
                                n_cross_validations= 5)

In [2]:
# Submit your automl run

r_run  = experiment.submit(automl_config, show_output = False)

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = r_run.get_output()
print(best_run)
print(fitted_model)
compute_target.delete()